# Hybrid model for pneumonia detection
## 1. Setup
### 1.1. Installing dependencies

In [ ]:
!pip install pennylane scikit-learn numpy scipy matplotlib pandas pennylane-qiskit kagglehub scikit-image seaborn pillow opencv-python torch torchvision pennylane-lightning-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 77.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 78.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 12.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 41.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━

In [2]:
from dataclasses import dataclass
import torch
import numpy as np
import random
import os

@dataclass
class ExperimentConfig:
    # Experiment Metadata
    project_name: str = "Hybrid_ResNet50_QNN_Pneumonia"
    seed: int = 6
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Data Paths
    data_root: str = "/home/mforgo/.cache/kagglehub/datasets/paultimothymooney/chest-xray-pneumonia/versions/2/chest_xray/"
    output_dir: str = "./results"
    
    # Classical Backbone
    backbone_name: str = "resnet50"  # Fixed naming consistencyd
    feature_dim: int = 2048          # 2048 for ResNet50
    
    # Quantum Components
    n_qubits: int = 6                # Determined by PCA components
    n_layers: int = 2
    encoding_method: str = "amplitude" # 'amplitude' or 'angle'
    
    # Training Hyperparams
    batch_size: int = 32
    learning_rate: float = 0.01
    epochs: int = 50
    patience: int = 10

    # Preprocessing
    reduction_method: str = "pca"    # 'pca' or 'lda'
    target_dims: int = 64             # Dimensionality after reduction

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🔒 Global seed set to {seed}")

CFG = ExperimentConfig()
seed_everything(CFG.seed)
os.makedirs(CFG.output_dir, exist_ok=True)

🔒 Global seed set to 6


In [3]:
import kagglehub

# Download latest version
CFG.data_root = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia") + "/chest_xray/"
print("Path to dataset files:", CFG.data_root)

Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.
Path to dataset files: /kaggle/input/chest-xray-pneumonia/chest_xray/


## 2. Hybrid model
### 2.1. Classical preprocessing

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os

def get_transforms(img_size=224):
    """
    Standard ImageNet normalization. 
    Using standard stats ensures the pre-trained ResNet works as intended.
    """
    return transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

def get_dataloaders(config):
    """
    Creates DataLoaders for Train/Test/Val using ImageFolder.
    This replaces manual os.listdir loops.
    """
    loaders = {}
    sets = ['train', 'test', 'val']
    
    print(f"📂 Loading data from: {config.data_root}")
    
    for split in sets:
        path = os.path.join(config.data_root, split)
        if not os.path.exists(path):
            print(f"⚠️ Warning: Split '{split}' not found at {path}")
            continue
            
        # ImageFolder automatically handles class labels based on folder names
        dataset = datasets.ImageFolder(root=path, transform=get_transforms())
        
        loaders[split] = DataLoader(
            dataset, 
            batch_size=config.batch_size, 
            shuffle=False, # Important: Keep False to match features with filenames later
            num_workers=2, # Parallel loading
            pin_memory=True
        )
        print(f"   • {split.upper()}: Found {len(dataset)} images")
        
    return loaders

class FeatureExtractor(nn.Module):
    """
    Wraps ResNet50 to output raw features instead of classification scores.
    """
    def __init__(self):
        super().__init__()
        # Load modern V2 weights for better performance
        weights = models.ResNet50_Weights.IMAGENET1K_V2
        self.backbone = models.resnet50(weights=weights)
        
        # Replace the final classification layer (fc) with Identity
        # This allows us to get the 2048 feature vector directly
        self.backbone.fc = nn.Identity()
        self.backbone.eval() # Set to evaluation mode (freezes BatchNorm)
        
    def forward(self, x):
        return self.backbone(x)

def run_feature_extraction(config):
    """
    The Engine: Loads data, passes it through ResNet, and saves features.
    """
    device = torch.device(config.device)
    model = FeatureExtractor().to(device)
    loaders = get_dataloaders(config)
    
    # Create directory for saved features
    save_dir = os.path.join(config.output_dir, "features")
    os.makedirs(save_dir, exist_ok=True)
    
    metadata = []
    
    print(f"\n🚀 Starting extraction with {config.backbone_name} on {device}...")
    
    with torch.no_grad(): # Disable gradient calculation for speed
        for split, loader in loaders.items():
            
            for batch_idx, (images, labels) in enumerate(tqdm(loader, desc=f"Extracting {split}")):
                images = images.to(device)
                
                # Forward pass: Get features (Batch_Size, 2048)
                with autocast():  # Mixed precision for speed
                    features = model(images)
                
                features = model(images).cpu().numpy()
                # Match features back to original filenames
                # We calculate the global index based on batch size
                start_idx = batch_idx * config.batch_size
                
                for i, feat in enumerate(features):
                    global_idx = start_idx + i
                    # Retrieve path from dataset.samples which is [(path, class_idx), ...]
                    original_path, label_idx = loader.dataset.samples[global_idx]
                    filename = os.path.basename(original_path)
                    classname = loader.dataset.classes[label_idx]
                    
                    # Save individual feature file
                    save_name = f"{split}_{classname}_{filename}.npy"
                    save_path = os.path.join(save_dir, save_name)
                    np.save(save_path, feat)
                    
                    metadata.append({
                        'feature_path': save_path,
                        'label': label_idx, # 0 or 1
                        'classname': classname,
                        'split': split,
                        'original_path': original_path
                    })
    
    # Save metadata CSV for easy loading later
    meta_path = os.path.join(config.output_dir, "metadata.csv")
    pd.DataFrame(metadata).to_csv(meta_path, index=False)
    print(f"✅ Extraction complete. Metadata saved to {meta_path}")
    return meta_path

# Execute the pipeline using our Config
meta_csv_path = run_feature_extraction(CFG)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 173MB/s]


📂 Loading data from: /kaggle/input/chest-xray-pneumonia/chest_xray/
   • TRAIN: Found 5216 images
   • TEST: Found 624 images
   • VAL: Found 16 images

🚀 Starting extraction with resnet50 on cuda...


Extracting train:   0%|          | 0/163 [00:00<?, ?it/s]

Extracting test:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting val:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Extraction complete. Metadata saved to ./results/metadata.csv


In [5]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split  
import joblib
import os

def load_features(config):
    """
    Loads features from .npy files based on the metadata CSV.
    """
    meta_path = os.path.join(config.output_dir, "metadata.csv")
    if not os.path.exists(meta_path):
        raise FileNotFoundError(f"Metadata CSV not found at {meta_path}. Run Step 3 first.")
    
    df = pd.read_csv(meta_path)
    splits = ['train', 'val', 'test']
    
    X, y = {}, {}
    
    print(f"📂 Loading raw features...")
    for split in splits:
        subset = df[df['split'] == split]
        features = [np.load(path) for path in subset['feature_path']]
        X[split] = np.vstack(features)
        y[split] = subset['label'].values
    
    return X, y

def fix_validation_split(X, y, config):
    """
    OPRAVA: Sloučí Train (5216) a Val (16) a vytvoří nové rozdělení 80/20.
    Tím získáme cca 1000 validačních snímků místo 16.
    """
    print("\n⚠️ Fixing small validation set issue...")
    
    # 1. Sloučení
    X_combined = np.concatenate([X['train'], X['val']])
    y_combined = np.concatenate([y['train'], y['val']])
    
    # 2. Nové rozdělení (stratify zajistí správný poměr tříd)
    X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(
        X_combined, y_combined, 
        test_size=0.2, 
        stratify=y_combined, 
        random_state=config.seed
    )
    
    # 3. Aktualizace slovníků
    X['train'], y['train'] = X_train_new, y_train_new
    X['val'], y['val']     = X_val_new, y_val_new
    
    print(f"   Original Val size: 16 -> New Val size: {len(X_val_new)}")
    print(f"   New Train size:    {len(X_train_new)}")
    return X, y

def build_pipeline(config):
    steps = [('scaler', StandardScaler())]
    
    if config.reduction_method == 'pca':
        steps.append(('reducer', PCA(n_components=config.target_dims, random_state=config.seed)))
    elif config.reduction_method == 'lda':
        steps.append(('reducer', LDA(n_components=min(config.target_dims, 1))))
    
    if config.encoding_method == 'amplitude':
        steps.append(('normalizer', Normalizer(norm='l2')))
    elif config.encoding_method == 'angle':
        steps.append(('minmax', MinMaxScaler(feature_range=(0, np.pi))))
        
    return Pipeline(steps)

def run_classical_preprocessing(config):
    # 1. Load Data
    X, y = load_features(config)
    
    # 2. FIX DATA SPLIT (Tohle je ta klíčová oprava)
    X, y = fix_validation_split(X, y, config)
    
    # 3. Build & Fit Pipeline
    pipeline = build_pipeline(config)
    print(f"\n🔄 Fitting {config.reduction_method.upper()} Pipeline...")
    
    pipeline.fit(X['train'], y['train'])
    
    # 4. Transform & Save
    X_processed = {
        'train': pipeline.transform(X['train']),
        'val':   pipeline.transform(X['val']),
        'test':  pipeline.transform(X['test'])
    }
    
    processed_dir = os.path.join(config.output_dir, "processed_data")
    os.makedirs(processed_dir, exist_ok=True)
    
    for split in ['train', 'val', 'test']:
        np.save(os.path.join(processed_dir, f"X_{split}.npy"), X_processed[split])
        np.save(os.path.join(processed_dir, f"y_{split}.npy"), y[split])
        
    pipeline_path = os.path.join(config.output_dir, "preprocessing_pipeline.joblib")
    joblib.dump(pipeline, pipeline_path)
    
    print(f"✅ Preprocessing complete. Data ready for Quantum Training.")
    return X_processed, y

# Spuštění opraveného kroku
X_data, y_data = run_classical_preprocessing(CFG)

📂 Loading raw features...

⚠️ Fixing small validation set issue...
   Original Val size: 16 -> New Val size: 1047
   New Train size:    4185

🔄 Fitting PCA Pipeline...
✅ Preprocessing complete. Data ready for Quantum Training.


In [ ]:
import pennylane as qml
from pennylane import numpy as pnp
import torch
import numpy as np
from tqdm.notebook import tqdm
import os
import json
import time
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# --- 1. Modular Quantum Layers ---

def get_device(config):
    """Creates a PennyLane device with GPU acceleration."""
    try:
        # This uses NVIDIA cuQuantum for massive speedups on A100/L4
        return qml.device("lightning.gpu", wires=config.n_qubits)
    except:
        # Fallback if GPU is not available
        return qml.device("default.qubit", wires=config.n_qubits)

def embedding_layer(features, config):
    """
    Encodes a BATCH of classical data into quantum states.
    """
    wires = range(config.n_qubits)
    
    if config.encoding_method == 'amplitude':
        # PennyLane will detect if features has a batch dimension (e.g., shape 32, 64)
        qml.AmplitudeEmbedding(features=features, wires=wires, normalize=True, pad_with=0.0)
        
    elif config.encoding_method == 'angle':
        qml.AngleEmbedding(features=features, wires=wires, rotation='Y')

def ansatz_layer(params, config):
    """
    Optimized Ansatz for batched inputs.
    """
    n_wires = config.n_qubits
    
    for l in range(config.n_layers):
        # Parameterized Rotations
        for w in range(n_wires):
            # Rot is compatible with batching when using broadcastable params
            qml.Rot(params[l, w, 0], params[l, w, 1], params[l, w, 2], wires=w)
            
        # Entangling Layer
        for w in range(n_wires):
            qml.CNOT(wires=[w, (w + 1) % n_wires])

# --- 2. The QNode Builder ---

def build_qnode(config):
    dev = get_device(config)
    
    @qml.qnode(dev, interface="autograd", diff_method="adjoint", cache=True)
    def qnode(inputs, params):
        embedding_layer(inputs, config)
        ansatz_layer(params, config)
        # When inputs is batched, this returns a batch of expectation values
        return qml.expval(qml.PauliZ(0))
        
    return qnode 

# --- 3. Training Engine ---

def train_quantum_model(config, X_data, y_data):
    """
    Main training loop using Autograd.
    """
    print(f"\n⚛️ Initializing Quantum Model ({config.n_qubits} Qubits, {config.n_layers} Layers)...")
    
    # Initialize QNode
    qnode = build_qnode(config)
    
    # Initialize Parameters (Random weights)
    # Shape matching our manual ansatz: (L, N_wires, 3)
    param_shape = (config.n_layers, config.n_qubits, 3)
    params = pnp.random.uniform(0, 2*np.pi, size=param_shape, requires_grad=True)
    
    # Optimizer
    opt = qml.AdamOptimizer(stepsize=config.learning_rate)
    
    # Cost Function (MSE)
    def cost_fn(params, x_batch, y_batch):
        # PASS THE WHOLE BATCH AT ONCE
        # preds will now have shape (batch_size,)
        preds = qnode(x_batch, params)
        
        # Transform labels: 0/1 -> -1/1 to match PauliZ output
        targets = pnp.array([1 if y == 1 else -1 for y in y_batch], requires_grad=False)
        
        # Mean Squared Error
        return pnp.mean((preds - targets) ** 2)

    # Tracking
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    best_val_loss = float('inf')
    patience_counter = 0
    best_params = None
    
    # Data Setup
    X_train, y_train = X_data['train'], y_data['train']
    X_val, y_val = X_data['val'], y_data['val']
    
    batch_size = config.batch_size
    n_batches = len(X_train) // batch_size
    
    print(f"🚀 Starting training for {config.epochs} epochs...")
    
    start_time = time.time()
    
    for epoch in range(config.epochs):
        # Shuffle
        perm = np.random.permutation(len(X_train))
        X_train = X_train[perm]
        y_train = y_train[perm]
        
        epoch_loss = 0
        
        # Batch Loop
        with tqdm(total=n_batches, desc=f"Epoch {epoch+1}/{config.epochs}", leave=False) as pbar:
            for i in range(n_batches):
                batch_idx = slice(i * batch_size, (i + 1) * batch_size)
                X_batch = X_train[batch_idx]
                y_batch = y_train[batch_idx]
                
                # Step
                params, loss = opt.step_and_cost(lambda p: cost_fn(p, X_batch, y_batch), params)
                epoch_loss += loss
                pbar.update(1)
                pbar.set_postfix({'loss': f"{loss:.4f}"})
        
        avg_train_loss = epoch_loss / n_batches
        
        # Validation
        val_loss = cost_fn(params, X_val, y_val)
        
        # Val Accuracy (threshold at 0.0 because PauliZ is [-1, 1])
        # Faster Validation in the training loop
        val_preds_raw = qnode(X_val, params) # Pass all validation data at once if memory allows
        val_preds = (val_preds_raw > 0).astype(int) 
        val_acc = accuracy_score(y_val, val_preds)
        
        history['train_loss'].append(float(avg_train_loss))
        history['val_loss'].append(float(val_loss))
        history['val_acc'].append(val_acc)
        
        print(f"   Epoch {epoch+1}: Train Loss {avg_train_loss:.4f} | Val Loss {val_loss:.4f} | Val Acc {val_acc:.4f}")
        
        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_params = params.copy()
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= config.patience:
            print(f"⏹ Early stopping at epoch {epoch+1}")
            break
            
    train_time = time.time() - start_time
    print(f"✅ Training done in {train_time:.1f}s. Best Val Loss: {best_val_loss:.4f}")
    
    return best_params, qnode, history

# --- 4. Evaluation Function ---

def evaluate_model(config, params, qnode, X_test, y_test):
    print("\n📊 Evaluating on Test Set...")
    preds_raw = np.array([qnode(x, params) for x in X_test])
    
    # Map [-1, 1] -> [0, 1]
    probs = (preds_raw + 1) / 2
    preds = (probs > 0.5).astype(int)
    
    metrics = {
        'accuracy': accuracy_score(y_test, preds),
        'f1_score': f1_score(y_test, preds, average='macro'),
        'auc_score': roc_auc_score(y_test, probs)
    }
    
    print(f"   Test Accuracy: {metrics['accuracy']:.2%}")
    print(f"   Test F1 Score: {metrics['f1_score']:.4f}")
    print(f"   Test AUC:      {metrics['auc_score']:.4f}")
    return metrics, preds, probs

# --- Execution ---

# 1. Train
best_params, qnode, history = train_quantum_model(CFG, X_data, y_data)

# 2. Evaluate
test_metrics, test_preds, test_probs = evaluate_model(CFG, best_params, qnode, X_data['test'], y_data['test'])

# 3. Save
results_path = os.path.join(CFG.output_dir, "quantum_results.json")
with open(results_path, 'w') as f:
    json.dump({
        'config': {k: str(v) for k, v in CFG.__dict__.items()},
        'metrics': test_metrics,
        'history': history
    }, f, indent=4)
np.save(os.path.join(CFG.output_dir, "best_params.npy"), best_params)
print(f"💾 Saved to {results_path}")


⚛️ Initializing Quantum Model (6 Qubits, 2 Layers)...
🚀 Starting training for 50 epochs...


/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


Epoch 1/50:   0%|          | 0/130 [00:00<?, ?it/s]

   Epoch 1: Train Loss 0.9357 | Val Loss 0.8241 | Val Acc 0.7421


Epoch 2/50:   0%|          | 0/130 [00:00<?, ?it/s]

...

Epoch 22/50:   0%|          | 0/130 [00:00<?, ?it/s]

   Epoch 22: Train Loss 0.6830 | Val Loss 0.6526 | Val Acc 0.7784


Epoch 23/50:   0%|          | 0/130 [00:00<?, ?it/s]

   Epoch 23: Train Loss 0.6811 | Val Loss 0.6546 | Val Acc 0.7612


Epoch 24/50:   0%|          | 0/130 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
import numpy as np
import os

# Nastavení profesionálního vzhledu grafů
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.dpi': 300}) # 300 DPI pro tiskovou kvalitu

def plot_training_history(history, save_dir):
    """
    Vykreslí vývoj chyby (Loss) a přesnosti (Accuracy) během tréninku.
    """
    epochs = range(1, len(history['train_loss']) + 1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. Graf Chyby (Loss)
    ax1.plot(epochs, history['train_loss'], 'b-', label='Trénovací chyba', linewidth=2)
    ax1.plot(epochs, history['val_loss'], 'r--', label='Validační chyba', linewidth=2)
    ax1.set_title('Vývoj chybové funkce (Loss)', fontweight='bold')
    ax1.set_xlabel('Epocha')
    ax1.set_ylabel('Loss (MSE)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Graf Přesnosti (Accuracy)
    ax2.plot(epochs, history['val_acc'], 'g-', label='Validační přesnost', linewidth=2)
    ax2.set_title('Vývoj přesnosti na validační sadě', fontweight='bold')
    ax2.set_xlabel('Epocha')
    ax2.set_ylabel('Přesnost (0-1)')
    ax2.set_ylim(0, 1.0)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_path = os.path.join(save_dir, "training_history.png")
    plt.savefig(save_path, bbox_inches='tight')
    print(f"📈 Graf tréninku uložen: {save_path}")
    plt.show()

def plot_evaluation_metrics(y_true, y_pred, y_probs, save_dir):
    """
    Vykreslí Matici záměn a ROC křivku.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. Matice záměn (Confusion Matrix)
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
                xticklabels=['Normal', 'Pneumonia'],
                yticklabels=['Normal', 'Pneumonia'],
                annot_kws={"size": 14, "weight": "bold"})
    ax1.set_title('Matice záměn (Confusion Matrix)', fontweight='bold')
    ax1.set_ylabel('Skutečná třída')
    ax1.set_xlabel('Predikovaná třída')
    
    # 2. ROC Křivka
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)
    
    ax2.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC křivka (AUC = {roc_auc:.2f})')
    ax2.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax2.set_xlim([0.0, 1.0])
    ax2.set_ylim([0.0, 1.05])
    ax2.set_xlabel('False Positive Rate (1 - Specificita)')
    ax2.set_ylabel('True Positive Rate (Senzitivita)')
    ax2.set_title('ROC Křivka', fontweight='bold')
    ax2.legend(loc="lower right")
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_path = os.path.join(save_dir, "evaluation_metrics.png")
    plt.savefig(save_path, bbox_inches='tight')
    print(f"📊 Grafy metrik uloženy: {save_path}")
    plt.show()

# --- Spuštění vizualizace ---

# Používáme proměnné z předchozích kroků:
# history (z kroku 5)
# y_data['test'] (z kroku 4/5 - skutečné hodnoty)
# test_preds (z kroku 5 - predikované 0/1)
# test_probs (z kroku 5 - pravděpodobnosti)

print(f"=== VIZUALIZACE VÝSLEDKŮ PRO SOČ ===")
plot_training_history(history, CFG.output_dir)
plot_evaluation_metrics(y_data['test'], test_preds, test_probs, CFG.output_dir)

# Bonus: Výpis finálních čísel pro text práce
cm = confusion_matrix(y_data['test'], test_preds)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print("\n📝 Data pro tabulku výsledků v SOČ:")
print(f"-----------------------------------")
print(f"Počet testovacích snímků: {len(y_data['test'])}")
print(f"TP (Správně Pneumonie):   {tp}")
print(f"TN (Správně Zdraví):      {tn}")
print(f"FP (Falešný poplach):     {fp}")
print(f"FN (Přehlédnutá nemoc):   {fn}")
print(f"-----------------------------------")
print(f"Senzitivita (Recall):     {sensitivity:.4f}")
print(f"Specificita:              {specificity:.4f}")
print(f"AUC:                      {roc_auc_score(y_data['test'], test_probs):.4f}")

In [ ]:
# ==========================
# 🔍 Single Image Evaluation
# ==========================
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import joblib
import pennylane as qml

print("=== Single Image Test Mode ===")
img_path = input("Enter path to an image (.jpeg/.jpg/.png): ").strip()
metadata_path = "./data/features/metadata.csv"

# --- Load metadata to identify label ---
meta = pd.read_csv(metadata_path)
meta['image_ref'] = meta['image_path'].astype(str).str.lower()
img_name = os.path.basename(img_path).lower()
true_label = None

for _, row in meta.iterrows():
    if img_name in os.path.basename(row['image_ref']):
        true_label = "PNEUMONIA" if int(row['label']) == 1 else "NORMAL"
        break

# --- Load and preprocess image ---
img = Image.open(img_path).convert('L').resize((224,224))
plt.imshow(img, cmap='gray')
plt.title(f"Input Image: {os.path.basename(img_path)}")
plt.axis('off')
plt.show()

from torchvision import models, transforms
import torch

# --- Load same CNN as used in feature extraction ---
model = models.resnet50(weights='IMAGENET1K_V2')
model = torch.nn.Sequential(*(list(model.children())[:-1]))  # remove final FC
model.eval()

# --- Define same preprocessing as training pipeline ---
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

# --- Run CNN feature extraction ---
img_rgb = Image.open(img_path).convert('RGB')
input_tensor = preprocess(img_rgb).unsqueeze(0)  # shape (1,3,224,224)
with torch.no_grad():
    features_cnn = model(input_tensor).squeeze().numpy()  # shape (2048,)

img_flat = features_cnn.reshape(1, -1)


# --- Load reducer from training (e.g., PCA joblib) ---
reducer_path = f"./results/{REDUCTION_METHOD}_reducer_{ACTUAL_DIMS}d.joblib"
if os.path.exists(reducer_path):
    reducer = joblib.load(reducer_path)
    img_reduced = reducer.transform(img_flat)
else:
    print(f"⚠️ Reducer not found at {reducer_path}, reusing mean/std from training.")
    img_reduced = img_flat[:, :ACTUAL_DIMS]

# --- Prepare for quantum encoding ---
if ENCODING == "amplitude":
    vec = np.zeros(2**N_QUBITS)
    vec[:len(img_reduced[0])] = img_reduced[0]
    q_input = vec / np.linalg.norm(vec)
else:
    from sklearn.preprocessing import MinMaxScaler
    sc = MinMaxScaler((0, 2*np.pi))
    q_input = sc.fit_transform(img_reduced)[0]

# --- Define quantum circuit ---
dev = qml.device("default.qubit", wires=N_QUBITS, shots=None)

@qml.qnode(dev)
def qnode_predict(x, theta):
    if ENCODING == "amplitude":
        qml.AmplitudeEmbedding(x, wires=range(N_QUBITS), normalize=True, pad_with=0.0)
    else:
        for i, val in enumerate(x):
            qml.RY(val, wires=i)
    p = theta.reshape(N_LAYERS, N_QUBITS, 3)
    for l in range(N_LAYERS):
        for w in range(N_QUBITS):
            qml.RX(p[l,w,0], wires=w)
            qml.RY(p[l,w,1], wires=w)
            qml.RZ(p[l,w,2], wires=w)
        for w in range(N_QUBITS):
            qml.CNOT(wires=[w, (w+1)%N_QUBITS])
    return qml.expval(qml.PauliZ(0))

# --- Load trained quantum parameters ---
param_path = f"./results/params_{ENCODING}_{REDUCTION_METHOD}{ACTUAL_DIMS}d_{N_LAYERS}L.npy"
if not os.path.exists(param_path):
    raise FileNotFoundError(f"Trained quantum parameters not found: {param_path}")
params_final = np.load(param_path)

# --- Predict ---
prediction = qnode_predict(q_input, params_final)
prob_pneumonia = (1 + prediction) / 2
pred_label = "PNEUMONIA" if prob_pneumonia > 0.5 else "NORMAL"

print("\n🧠 Model Prediction:")
print(f"  → Predicted: {pred_label} (probability={prob_pneumonia:.3f})")
if true_label:
    print(f"  → Actual Label: {true_label}")
    print("✅ Correct!" if pred_label == true_label else "❌ Incorrect.")

plt.title(f"Prediction: {pred_label}  |  Actual: {true_label or 'Unknown'}")
plt.axis('off')
plt.imshow(img, cmap='gray')
plt.show()
